## Detecção de movimentos

In [ ]:
import cv2
import mediapipe as mp
import os
from tqdm import tqdm
import math
from collections import deque

class PoseClassifier:
    """
    Classe para classificar poses e contar movimentos usando landmarks do MediaPipe.
    """
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        
        # --- Para Agachamento ---
        self.current_squat_state = "standing" # "standing" ou "squatting"
        self.squats_count = 0

        # --- Para Levantamento de Braco ---
        self.left_arm_up_state = False
        self.right_arm_up_state = False
        self.left_arm_raise_count = 0
        self.right_arm_raise_count = 0

        # --- Para T-Pose ---
        self.t_pose_state = False # True quando em T-Pose
        self.t_pose_count = 0

        # --- Para detecção de Concordancia (Nodding) ---
        self.prev_nose_y_relative = None # Posição Y do nariz relativa aos ombros no frame anterior
        self.nod_state = "neutral" # "neutral", "down", "up"
        self.nods_count = 0
        self.NOD_THRESHOLD = 0.015 # Deslocamento Y relativo para detectar um nod (ajuste)
        self.is_nodding_active = False # Flag para indicar se o movimento está ativo no frame atual

        # --- Para detecção de Discordancia (Shaking) ---
        self.prev_nose_x_relative = None # Posição X do nariz relativa aos ombros no frame anterior
        self.shake_state = "neutral" # "neutral", "left", "right"
        self.shakes_count = 0
        self.SHAKE_THRESHOLD = 0.015 # Deslocamento X relativo para detectar um shake (ajuste)
        self.is_shaking_active = False # Flag para indicar se o movimento está ativo no frame atual

        # --- Para detecção de Caminhada (Walking) ---
        self.left_ankle_relative_y_history = deque(maxlen=5) # Historico para suavizar pos Y do tornozelo esquerdo
        self.right_ankle_relative_y_history = deque(maxlen=5) # Historico para suavizar pos Y do tornozelo direito
        self.left_step_state = "down" # "up", "down"
        self.right_step_state = "down" # "up", "down"
        self.walking_steps_count = 0
        self.WALK_VERTICAL_MOVEMENT_THRESHOLD = -0.04 # Quanto o tornozelo precisa subir (y menor) para ser "up" (ajuste)
        self.is_walking = False # Flag para indicar se a pessoa está caminhando ativamente
        self.walking_frames_no_step = 0 # Contador para desativar is_walking após inatividade
        self.WALK_INACTIVITY_THRESHOLD = 30 # Frames sem detecção de passo para parar de considerar "walking"
        
        # --- Para detecção de Posições Anômalas ---
        self.previous_landmarks = None # Armazena landmarks do frame anterior para calcular delta de movimento
        self.anomalous_count = 0
        self.anomalous_event_active = False # True se uma anomalia esta ativa no momento
        self.ANOMALOUS_MOVEMENT_THRESHOLD = 0.8 # Limiar para detectar um movimento brusco (ajuste)

        # --- Flags para exibir status no frame atual ---
        self.current_primary_pose_label = "Posicao Desconhecida"

    def calculate_angle(self, a, b, c):
        """
        Calcula o ângulo em graus entre três pontos (a, b, c) com o vértice em b.
        Os pontos devem ser objetos de landmark do MediaPipe (x, y, z).
        """
        try:
            a_coords = [a.x, a.y]
            b_coords = [b.x, b.y]
            c_coords = [c.x, c.y]

            ba = [a_coords[0] - b_coords[0], a_coords[1] - b_coords[1]]
            bc = [c_coords[0] - b_coords[0], c_coords[1] - b_coords[1]]

            dot_product = ba[0] * bc[0] + ba[1] * bc[1]
            magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
            magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)

            if magnitude_ba == 0 or magnitude_bc == 0:
                return 0

            angle_rad = math.acos(min(max(dot_product / (magnitude_ba * magnitude_bc), -1.0), 1.0))
            angle_deg = math.degrees(angle_rad)
            return angle_deg
        except Exception:
            return 0

    def update_states(self, current_landmarks):
        """
        Atualiza os estados internos de classificação e contadores com base nos landmarks atuais.
        """
        # --- Resetar flags de deteccao para o frame atual ---
        self.current_primary_pose_label = "Posicao Desconhecida"
        self.is_nodding_active = False
        self.is_shaking_active = False
        # self.is_walking eh um estado mais persistente e tratado abaixo
        
        # O estado anomalous_event_active so eh resetado se a anomalia nao estiver mais acontecendo.
        # Ele controla se o contador ja foi incrementado para o evento atual.

        if not current_landmarks: # Apenas verifica se ha landmarks
            # Se nao ha landmarks detectados, reseta todos os estados para evitar falsos positivos
            self.prev_nose_y_relative = None
            self.prev_nose_x_relative = None
            self.nod_state = "neutral"
            self.shake_state = "neutral"

            self.left_ankle_relative_y_history.clear()
            self.right_ankle_relative_y_history.clear()
            self.left_step_state = "down"
            self.right_step_state = "down"
            self.is_walking = False
            self.walking_frames_no_step = 0
            
            self.previous_landmarks = None # Reseta landmarks anteriores para evitar salto apos reaparicao
            self.anomalous_event_active = False # Reseta estado de anomalia
            return

        # --- Calcular Magnitude de Movimento para Deteccao de Anomalias ---
        current_movement_magnitude = 0.0
        if self.previous_landmarks and current_landmarks:
            total_movement = 0.0
            # Inclui as principais articulacoes para detectar movimento geral do corpo
            landmark_indices = [
                self.mp_pose.PoseLandmark.LEFT_SHOULDER.value, self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
                self.mp_pose.PoseLandmark.LEFT_ELBOW.value, self.mp_pose.PoseLandmark.RIGHT_ELBOW.value,
                self.mp_pose.PoseLandmark.LEFT_WRIST.value, self.mp_pose.PoseLandmark.RIGHT_WRIST.value,
                self.mp_pose.PoseLandmark.LEFT_HIP.value, self.mp_pose.PoseLandmark.RIGHT_HIP.value,
                self.mp_pose.PoseLandmark.LEFT_KNEE.value, self.mp_pose.PoseLandmark.RIGHT_KNEE.value,
                self.mp_pose.PoseLandmark.LEFT_ANKLE.value, self.mp_pose.PoseLandmark.RIGHT_ANKLE.value,
                self.mp_pose.PoseLandmark.NOSE.value
            ]
            for i in landmark_indices:
                if i < len(current_landmarks) and i < len(self.previous_landmarks):
                    prev_lm = self.previous_landmarks[i]
                    curr_lm = current_landmarks[i]
                    # Calcula a distancia euclidiana no plano XY
                    distance = math.sqrt((curr_lm.x - prev_lm.x)**2 + (curr_lm.y - prev_lm.y)**2)
                    total_movement += distance
            current_movement_magnitude = total_movement
        
        self.previous_landmarks = current_landmarks # Atualiza landmarks anteriores para o proximo frame

        # Obter todos os landmarks e calcular angulos necessarios UMA VEZ
        try:
            # Landmarks da cabeca e ombros para nod/shake
            nose = current_landmarks[self.mp_pose.PoseLandmark.NOSE.value]
            left_shoulder = current_landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = current_landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            
            # Landmarks para agachamento/bracos/t-pose
            left_hip = current_landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value]
            right_hip = current_landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP.value]
            left_elbow = current_landmarks[self.mp_pose.PoseLandmark.LEFT_ELBOW.value]
            right_elbow = current_landmarks[self.mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            left_wrist = current_landmarks[self.mp_pose.PoseLandmark.LEFT_WRIST.value]
            right_wrist = current_landmarks[self.mp_pose.PoseLandmark.RIGHT_WRIST.value]
            left_knee = current_landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value]
            right_knee = current_landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE.value]
            left_ankle = current_landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value]
            right_ankle = current_landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE.value]

            # Ponto de referencia para a linha dos ombros (para detecao de cabeca)
            shoulder_mid_y = (left_shoulder.y + right_shoulder.y) / 2
            shoulder_mid_x = (left_shoulder.x + right_shoulder.x) / 2

            # --- Deteccao de Concordancia (Nodding) ---
            # Posicao Y do nariz relativa ao meio dos ombros
            relative_nose_y = nose.y - shoulder_mid_y
            if self.prev_nose_y_relative is not None:
                delta_y = relative_nose_y - self.prev_nose_y_relative
                if abs(delta_y) > self.NOD_THRESHOLD:
                    if delta_y > 0 and self.nod_state == "neutral": # Cabeca inclinando para baixo
                        self.nod_state = "down"
                        self.is_nodding_active = True
                    elif delta_y < 0 and self.nod_state == "down": # Cabeca subindo depois de ir para baixo
                        self.nod_state = "up"
                        self.nods_count += 1
                        self.is_nodding_active = True
                    elif delta_y > 0 and self.nod_state == "up": # Cabeca descendo depois de ir para cima
                        self.nod_state = "down"
                        self.is_nodding_active = True
                else: # Se o movimento para, reseta o estado para neutral
                    self.nod_state = "neutral"
            self.prev_nose_y_relative = relative_nose_y

            # --- Deteccao de Discordancia (Shaking) ---
            # Posicao X do nariz relativa ao meio dos ombros
            relative_nose_x = nose.x - shoulder_mid_x
            if self.prev_nose_x_relative is not None:
                delta_x = relative_nose_x - self.prev_nose_x_relative
                if abs(delta_x) > self.SHAKE_THRESHOLD:
                    if delta_x < 0 and self.shake_state == "neutral": # Cabeca inclinando para a esquerda
                        self.shake_state = "left"
                        self.is_shaking_active = True
                    elif delta_x > 0 and self.shake_state == "left": # Cabeca indo para a direita depois de ir para a esquerda
                        self.shake_state = "right"
                        self.shakes_count += 1
                        self.is_shaking_active = True
                    elif delta_x < 0 and self.shake_state == "right": # Cabeca indo para a esquerda depois de ir para a direita
                        self.shake_state = "left"
                        self.is_shaking_active = True
                else: # Se o movimento para, reseta o estado para neutral
                    self.shake_state = "neutral"
            self.prev_nose_x_relative = relative_nose_x

            # --- Deteccao de Caminhada (Walking) ---
            # Calcular posicoes Y dos tornozelos relativas aos quadris correspondentes
            relative_left_ankle_to_hip = left_ankle.y - left_hip.y
            relative_right_ankle_to_hip = right_ankle.y - right_hip.y

            # Adicionar ao historico para suavizacao
            self.left_ankle_relative_y_history.append(relative_left_ankle_to_hip)
            self.right_ankle_relative_y_history.append(relative_right_ankle_to_hip)

            # Usar media do historico para deteccao de levantamento do pe
            if len(self.left_ankle_relative_y_history) == self.left_ankle_relative_y_history.maxlen:
                avg_left_ankle_y_rel = sum(self.left_ankle_relative_y_history) / len(self.left_ankle_relative_y_history)
                avg_right_ankle_y_rel = sum(self.right_ankle_relative_y_history) / len(self.right_ankle_relative_y_history)

                # Deteccao para o pe esquerdo
                if avg_left_ankle_y_rel < self.WALK_VERTICAL_MOVEMENT_THRESHOLD and self.left_step_state == "down":
                    self.left_step_state = "up"
                    self.is_walking = True
                    self.walking_frames_no_step = 0
                elif avg_left_ankle_y_rel >= self.WALK_VERTICAL_MOVEMENT_THRESHOLD and self.left_step_state == "up":
                    self.left_step_state = "down"
                    self.walking_steps_count += 1
                    self.is_walking = True
                    self.walking_frames_no_step = 0
                
                # Deteccao para o pe direito
                if avg_right_ankle_y_rel < self.WALK_VERTICAL_MOVEMENT_THRESHOLD and self.right_step_state == "down":
                    self.right_step_state = "up"
                    self.is_walking = True
                    self.walking_frames_no_step = 0
                elif avg_right_ankle_y_rel >= self.WALK_VERTICAL_MOVEMENT_THRESHOLD and self.right_step_state == "up":
                    self.right_step_state = "down"
                    self.walking_steps_count += 1
                    self.is_walking = True
                    self.walking_frames_no_step = 0
            
            # Logica para detectar o fim da caminhada
            if self.is_walking:
                self.walking_frames_no_step += 1
                if self.walking_frames_no_step > self.WALK_INACTIVITY_THRESHOLD:
                    self.is_walking = False
                    self.walking_frames_no_step = 0

            # --- Classificacoes de Poses Individuais ---
            # Prioridade: Agachamento > T-Pose > Braco Levantado
            
            # 1. Agachamento (Squat)
            SQUAT_ANGLE_THRESHOLD = 100
            STAND_ANGLE_THRESHOLD = 160
            
            # Calcular angulos para as poses (so se nao houver um erro antes)
            left_arm_angle = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_knee_angle = self.calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = self.calculate_angle(right_hip, right_knee, right_ankle)

            is_squatting_frame = (left_knee_angle < SQUAT_ANGLE_THRESHOLD and right_knee_angle < SQUAT_ANGLE_THRESHOLD and 
                                    left_hip.y > left_knee.y and right_hip.y > right_knee.y)

            if is_squatting_frame:
                if self.current_squat_state == "standing":
                    self.current_squat_state = "squatting"
                self.current_primary_pose_label = "Agachamento"
            elif self.current_squat_state == "squatting" and \
                    left_knee_angle > STAND_ANGLE_THRESHOLD and right_knee_angle > STAND_ANGLE_THRESHOLD:
                self.squats_count += 1
                self.current_squat_state = "standing"
            else:
                if self.current_squat_state == "squatting":
                    self.current_primary_pose_label = "Agachamento"
                else:
                    self.current_squat_state = "standing"

            # 2. T-Pose
            if self.current_primary_pose_label == "Posicao Desconhecida": # Só classifica se nenhuma outra pose principal foi detectada
                T_POSE_ARM_ANGLE_THRESHOLD = 160
                T_POSE_Y_DIFF_THRESHOLD = 0.05

                arms_straight = left_arm_angle > T_POSE_ARM_ANGLE_THRESHOLD and right_arm_angle > T_POSE_ARM_ANGLE_THRESHOLD
                wrists_at_shoulder_height = (abs(left_wrist.y - left_shoulder.y) < T_POSE_Y_DIFF_THRESHOLD) and \
                                            (abs(right_wrist.y - right_shoulder.y) < T_POSE_Y_DIFF_THRESHOLD)
                wrists_x_distance_ok = (left_wrist.x < left_shoulder.x and right_wrist.x > right_shoulder.x)
                
                if arms_straight and wrists_at_shoulder_height and wrists_x_distance_ok:
                    if not self.t_pose_state:
                        self.t_pose_count += 1
                        self.t_pose_state = True
                    self.current_primary_pose_label = "T-Pose"
                else:
                    self.t_pose_state = False

            # 3. Braco Levantado (Left/Right/Both)
            if self.current_primary_pose_label == "Posicao Desconhecida": # Só classifica se nenhuma outra pose principal foi detectada
                Y_OFFSET_THRESHOLD = 0.1 
                left_arm_raised = left_wrist.y < left_shoulder.y - Y_OFFSET_THRESHOLD
                right_arm_raised = right_wrist.y < right_shoulder.y - Y_OFFSET_THRESHOLD
                
                if left_arm_raised and right_arm_raised:
                    self.current_primary_pose_label = "Ambos Bracos Levantados"
                    if not self.left_arm_up_state: # Verifica se ambos estavam pra baixo
                        self.left_arm_raise_count += 1 # Contagem única para o movimento
                        self.right_arm_raise_count += 1
                    self.left_arm_up_state = True
                    self.right_arm_up_state = True
                elif left_arm_raised:
                    self.current_primary_pose_label = "Braco Esquerdo Levantado"
                    if not self.left_arm_up_state:
                        self.left_arm_raise_count += 1
                    self.left_arm_up_state = True
                    self.right_arm_up_state = False # Garante que o outro braço não está marcado como levantado
                elif right_arm_raised:
                    self.current_primary_pose_label = "Braco Direito Levantado"
                    if not self.right_arm_up_state:
                        self.right_arm_raise_count += 1
                    self.right_arm_up_state = True
                    self.left_arm_up_state = False # Garante que o outro braço não está marcado como levantado
                else:
                    self.left_arm_up_state = False
                    self.right_arm_up_state = False
            
            # --- Deteccao de Anomalias ---
            # Deve ser verificada depois de todas as outras deteccoes especificas
            # para garantir que nao estamos contando movimentos "normais" como anomalias.
            is_any_specific_movement_active = (
                self.is_nodding_active or
                self.is_shaking_active or
                self.is_walking or
                (self.current_squat_state == "squatting") or # Ativo agachando
                (self.t_pose_state) # Em T-Pose
            )

            # Se ha um movimento brusco E nao estamos em nenhuma das acoes especificas
            # E a pose principal ainda nao foi classificada por uma pose especifica mais estatica
            if current_movement_magnitude > self.ANOMALOUS_MOVEMENT_THRESHOLD and \
               not is_any_specific_movement_active and \
               self.current_primary_pose_label == "Posicao Desconhecida":
                
                if not self.anomalous_event_active:
                    self.anomalous_count += 1
                    self.anomalous_event_active = True
                self.current_primary_pose_label = "Anomalia Detectada" # Prioriza esta label
            else:
                self.anomalous_event_active = False # Reseta se a anomalia nao esta mais ativa

        except Exception as e:
            # print(f"Aviso: Erro na classificacao da pose: {e}. Alguns landmarks podem estar ausentes.")
            pass


def detect_pose_and_count_movements(input_video_path, output_video_movimentos_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print(f"Erro: Nao foi possivel abrir o video em {input_video_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_movimentos_path, fourcc, fps, (width, height))

    pose_classifier = PoseClassifier()

    frame_count = 0 # Inicializa o contador de frames

    for _ in tqdm(range(total_frames), desc="Processando video"):
        ret, frame = cap.read()

        if not ret:
            break

        frame_count += 1 # Incrementa o contador de frames a cada iteração

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb_frame.flags.writeable = False

        results = pose.process(rgb_frame)
        
        rgb_frame.flags.writeable = True

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            pose_classifier.update_states(results.pose_landmarks.landmark)
        else:
            # Se nao ha landmarks, chama update_states com None para resetar os estados
            pose_classifier.update_states(None)

        # --- Exibir a classificacao e as contagens no frame ---
        # Exibir o numero do frame analisado na parte inferior esquerda, em preto
        cv2.putText(frame, f'Frame: {frame_count}', (10, height - 30), # Posição (x, y) para o canto inferior esquerdo
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA) # Cor preta (0,0,0)
        
        # As demais informacoes de pose e movimento (posições ajustadas)
        cv2.putText(frame, f'Pose Principal: {pose_classifier.current_primary_pose_label}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Agachamentos: {pose_classifier.squats_count}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Braco Esquerdo Levantado: {pose_classifier.left_arm_raise_count}', (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Braco Direito Levantado: {pose_classifier.right_arm_raise_count}', (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f'T-Poses: {pose_classifier.t_pose_count}', (10, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Concordancia: {pose_classifier.nods_count} {"(Ativo)" if pose_classifier.is_nodding_active else ""}', (10, 180),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (120, 20, 220), 2, cv2.LINE_AA) # Roxa
        cv2.putText(frame, f'Discordancia: {pose_classifier.shakes_count} {"(Ativo)" if pose_classifier.is_shaking_active else ""}', (10, 210),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (20, 220, 120), 2, cv2.LINE_AA) # Verde Agua
        cv2.putText(frame, f'Passos: {pose_classifier.walking_steps_count} {"(Caminhando)" if pose_classifier.is_walking else ""}', (10, 240),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 100, 0), 2, cv2.LINE_AA) # Azul
        
        # Nova linha para o contador de Anomalias
        cv2.putText(frame, f'Anomalias: {pose_classifier.anomalous_count} {"(Ativo)" if pose_classifier.anomalous_event_active else ""}', (10, 270),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA) # Vermelho para Anomalias


        out.write(frame)

        cv2.imshow('Video de Classificacao de Movimento', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Caminho para o video de entrada e saida
script_dir = os.getcwd()
input_video_path = os.path.join(script_dir, 'Unlocking Facial Recognition_ Diverse Activities Analysis.mp4')
output_video_movimentos_path = os.path.join(script_dir, 'output_video_reconhecimento_movimentos.mp4')

print(f"Processando video: {input_video_path}")
detect_pose_and_count_movements(input_video_path, output_video_movimentos_path)
print(f"Video processado e salvo em: {output_video_movimentos_path}")

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

I0000 00:00:1767904573.643443 14573762 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processando video: /Users/luizviana/tech_chall_4/Unlocking Facial Recognition_ Diverse Activities Analysis.mp4


Processando video:   0%|          | 0/3326 [00:00<?, ?it/s]W0000 00:00:1767904573.727617 14573934 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767904573.740829 14573934 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1767904573.754459 14573934 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Processando video: 100%|██████████| 3326/3326 [01:46<00:00, 31.31it/s]

Video processado e salvo em: /Users/luizviana/tech_chall_4/output_video_reconhecimento_movimentos_v5.mp4
